## 시작하기 (가장 빠른 루틴)

In [8]:
import gzip, pickle
import yacht_dp_opt_fastpath as y

# 1) 슬림 캐시 로드(있으면 강력추천)
with gzip.open("yacht_dp_cache_slim.pkl.gz","rb") as f:
    d = pickle.load(f)
y.Q_MEMO.update(d.get("Q", {}))
y.EV0_VEC.update(d.get("EV0", {}))

# 2) 바로 질의
dice = [1,1,2,5,4]
print("best hold =",y.choose_best_hold(dice, y.CATS, upper_sum=0, r=2))     # 이번 턴: 무엇을 들고 갈지
print("best category =",y.choose_best_category_final(dice, y.CATS, upper_sum=0))# 이번 턴: 어느 칸에 적을지

best hold = {'keep_hist': (0, 0, 0, 0, 1, 0), 'keep_example': [5], 'reroll_count': 4, 'expected_value': 185.74122410217865}
best category = {'best_category': 'Aces', 'total_value': 179.63396678834627, 'immediate': 2, 'future': 177.63396678834627, 'ranked': [{'category': 'Aces', 'total': 179.63396678834627, 'immediate': 2, 'future': 177.63396678834627}, {'category': 'FourKind', 'total': 175.12224337769928, 'immediate': 0, 'future': 175.12224337769928}, {'category': 'Yacht', 'total': 173.05067305025875, 'immediate': 0, 'future': 173.05067305025875}, {'category': 'Twos', 'total': 172.60653333454118, 'immediate': 2, 'future': 170.60653333454118}, {'category': 'Choice', 'total': 169.39758296043667, 'immediate': 13, 'future': 156.39758296043667}, {'category': 'FullHouse', 'total': 169.336808189781, 'immediate': 0, 'future': 169.336808189781}, {'category': 'SmallStraight', 'total': 167.89332979424694, 'immediate': 0, 'future': 167.89332979424694}, {'category': 'LargeStraight', 'total': 165.48

## choose_best_hold(...) 출력 해석 (리롤 의사결정)

```python
{
  "keep_hist": (a,b,c,d,e,f),
  "keep_example": [ ... ],
  "reroll_count": n,
  "expected_value": EV
}

- **best_category**: **지금 이 눈**으로 **어느 칸에 적는 게 최적인지**.
- **immediate**: 지금 눈을 그 칸에 적으면 **즉시 들어오는 점수**(예: Aces면 1의 개수 × 1).
- **future**: 그 칸을 **소비한 이후**, **남은 칸들**로 **게임 끝까지 최적으로 플레이**했을 때의 **향후 기대 총점**.
  - 상단 보너스(+35)도 **조건 충족 시 자동 포함**됨. (예: 이 선택으로 상단합이 63을 넘으면 **future에 +35가 즉시 반영**)
- **total_value = immediate + future**: **이번 선택의 전체 기대값(EV)**.
   → **이 값이 큰 카테고리를 고르면 최적**.
- **ranked**: 모든 후보 카테고리를 **총합 EV 내림차순**으로 정렬한 표.

    각 항목에 즉시점수(**immediate**)와 향후가치(**future**)가 분해되어 있으니 **왜 이 선택이 좋은지** 파악하기 쉬움.

> 한 줄로: **어느 칸에 적어라 / 지금 점수는 x점 / 남은 판 기대는 y점 / 합쳐서 EV점.**

## choose_best_category_final(...) 출력 해석 (이번 턴 기록 의사결정)

```python
{
  "best_category": "…",
  "total_value": EV,
  "immediate": x,
  "future": y,
  "ranked": [
     {"category":"…","total":EV_i,"immediate":x_i,"future":y_i}, …
  ]
}

- best_category: 지금 이 눈으로 어느 칸에 적는 게 최적인지.
- immediate: 지금 눈을 그 칸에 적으면 즉시 들어오는 점수(예: Aces면 1의 개수 × 1).
- future: 그 칸을 소비한 이후, 남은 칸들로 게임 끝까지 최적으로 플레이했을 때의 향후 기대 총점.
    - 상단 보너스(+35)도 조건 충족 시 자동 포함됨. (예: 이 선택으로 상단합이 63을 넘으면 future에 +35가 즉시 반영)
- total_value = immediate + future: 이번 선택의 전체 기대값(EV).

  → 이 값이 큰 카테고리를 고르면 최적.
- ranked: 모든 후보 카테고리를 총합 EV 내림차순으로 정렬한 표.

  각 항목에 즉시점수(immediate)와 향후가치(future)가 분해되어 있으니 왜 이 선택이 좋은지 파악하기 쉬움.

한 줄로: 어느 칸에 적어라 / 지금 점수는 x점 / 남은 판 기대는 y점 / 합쳐서 EV점.

## 짧은 예시 ① (카테고리 결정)

- 상태: `dice=[1,1,1,1,4]`, `remaining=["Aces","Yacht"]`, `upper_sum=60`
- 결과(요지):
  - **best_category**: `"Aces"`
  - **immediate**: `4` (1이 4개)
  - **future**: `≈ 37.26`
     (설명: 이 선택으로 상단합이 63을 **넘어 보너스 +35 확정**, 남은 `"Yacht"`를 새 턴에서 r=2로 시도한 기대 ≈ `2.2647` → `35 + 2.2647`)
  - **total_value**: `≈ 41.26`
- 포인트: **즉시점수(Choice 등)가 더 커 보여도**, **나중에 남는 칸의 가치**(future)까지 합치면 Aces가 최적일 수 있다.

## 짧은 예시 ② (들고 가기 결정)

- 상태: `dice=[4,4,4,4,1]`, `remaining=["Yacht"]`, `r=2`
- 결과(요지):
  - **keep_hist**: `(0,0,0,4,0,0)` → **4 네 개 들고** `1개 리롤`
  - **reroll_count**: `1`
  - **expected_value**: `50 × (1 - (5/6)^2)` (+ 보너스가 이미 확정이면 여기 더해짐)

## 자주 헷갈리는 점 Q&A

- **Q. EV는 확률인가요?**

  **아니요.** 전부 **점수의 기대값**입니다. 단위는 **점**(points).
- **Q. 과거에 벌어둔 점수는 포함되나요?**

  **의사결정엔 불필요**해서 포함하지 않습니다(모든 선택에 공통).

  EV는 **현 시점부터 게임 종료까지의 기대 총점**으로 비교합니다.
- **Q. 상단 보너스(+35)는 어디에 더해지나요?**

  조건 충족 시 **future/expected_value에 자동 반영**됩니다.

  (특히 **마지막 칸이 하단 단일**일 때는 보너스가 **즉시 가산**되도록 처리되어 있음)
- **Q. `ranked`는 어떻게 쓰나요?**

  최적 선택 외에 **차선/차차선**까지 **즉시점수 vs 향후가치**를 비교해볼 때 유용합니다.

  (예: “지금 Choice가 17점이라 달콤하지만, 나중에 Choice를 남겼을 때 future가 훨씬 커서 총합이 밀림” 같은 통찰)